In [1]:
import os
import subprocess
import time
import sys
import pandas as pd
import random
module_path = "/data/luojaa/eukgen/"
if module_path not in sys.path:
    sys.path.append(module_path)
    
from core_functions.helper_functions import swarm_submit_with_lock, fasta_to_dict, dict_to_fasta
from paths_and_parameters import exe_reannotation, proteinfer_swarm_opts

In [2]:
proteinfer_swarmfile = "/data/luojaa/proteinfer/benchmark/output/tmp/reannotate_proteinfer.swarm"
clean_swarmfile = "/data/luojaa/CLEAN/app/results/tmp/reannotate_clean.swarm"
root = "/data/luojaa/CLEAN/app/"

In [3]:
ordered_infiles = []
with open(proteinfer_swarmfile, "r") as f:
    lines = f.readlines()
    for line in lines[6:]:
        ordered_infiles.append(line.strip().split(" ")[4].split("/")[-1].strip(".fasta"))

In [4]:
# print("\n".join(ordered_infiles))

In [5]:
proteinfer_swarm_opts

{'threads-per-process': 8,
 'gb-per-process': '10',
 'time': '72:00:00',
 'logdir': '/data/luojaa/log/swarm_out/',
 'job-name': 'proteinfer',
 'maxrunning': 1000}

In [6]:
proteinfer_swarm_opts['job-name'] = "clean"
proteinfer_swarm_opts['gb-per-process'] = "30"

In [7]:
# # format swarm file
# print("formatting swarm file")
# swarm = open(clean_swarmfile, 'w')

# # format swarm header for submission
# swarm.write(''.join([f'#SWARM --{key} {value}\n' for key, value in proteinfer_swarm_opts.items()]))

# for infile in ordered_infiles:
#     clean_cmd = f"python -u /data/luojaa/CLEAN/app/CLEAN_infer_fasta.py --fasta_data {infile} \n"
#     swarm.write(clean_cmd)
    
# swarm.close()
# #os.system(f"rm {root_fastas}*tmp*")

### parse output

In [8]:
proteinfer_swarmfile = "/data/luojaa/proteinfer/benchmark/output/tmp/reannotate_proteinfer.swarm"
ordered_infiles = []
with open(proteinfer_swarmfile, "r") as f:
    lines = f.readlines()
    for line in lines[6:]:
        ordered_infiles.append(line.strip().split(" ")[4].split("/")[-1].strip(".fasta"))

In [9]:
def parse_annot(file):
    try:
        data = pd.read_csv(file, sep=',', index_col=0, names=["protein_acc", "ec1", "ec2", "ec3", "ec4", "ec5", "ec6", "ec7", "ec8", "ec9", "ec10"]).dropna(axis = 1, how = "all")
    except pd.errors.ParserError:
        data = pd.read_csv(file, sep=',', index_col=0, names=["protein_acc", "ec1"])
    data["ec"] = [mappings[0].replace("EC:", "") for mappings in data["ec1"].str.split("/")]
    data["confidence_ec1"] = [float(mappings[1]) for mappings in data["ec1"].str.split("/")]
#     data[['label_type', 'predicted_label']] = [label.split(':') for label in data.predicted_label]

#     data_pfam = data[data.label_type == 'Pfam']
#     data_EC = data[data.label_type == 'EC']
#     data_GO = data[data.label_type == 'GO']
    
#     data_EC = data_EC.reset_index().drop_duplicates('sequence_name', keep='last').set_index('sequence_name')
#     data_EC['defined_EC'] = [True if label[-1] != '-' else False for label in data_EC.predicted_label]
    
    return data

In [10]:
output_csvs = [f"/data/luojaa/CLEAN/app/results/inputs/{file}_maxsep.csv" for file in ordered_infiles]


In [11]:
output_csvs[120]

'/data/luojaa/CLEAN/app/results/inputs/M00004.K00615_prok_maxsep.csv'

In [15]:
tophit = list(parse_annot(output_csvs[120]).value_counts("ec", normalize = True).reset_index().iloc[0])
confidence = parse_annot(output_csvs[120]).groupby("ec")["confidence_ec1"].mean()[tophit[0]]

In [18]:
tophit

['2.2.1.1', 0.59]

In [20]:
parse_annot(output_csvs[1])

,ec1,ec2,ec3,ec,confidence_ec1
protein_acc,,,,,
A0A1Y0HT15,EC:3.1.3.88/0.0000,NaN,NaN,3.1.3.88,0.0000
I0BQW4,EC:3.1.3.88/0.0284,NaN,NaN,3.1.3.88,0.0284
A0A142XJM9,EC:3.1.3.88/0.0029,NaN,NaN,3.1.3.88,0.0029
Q1CMX2,EC:3.1.3.3/0.0569,EC:3.1.3.73/0.0563,NaN,3.1.3.3,0.0569
A0AA96RCQ0,EC:3.1.3.88/0.0482,NaN,NaN,3.1.3.88,0.0482
...,...,...,...,...,...
A0A3S4SBG7,EC:3.1.3.73/0.0003,NaN,NaN,3.1.3.73,0.0003
A0A9E9P3D9,EC:3.1.3.3/0.0012,EC:5.4.2.11/0.0007,NaN,3.1.3.3,0.0012
A0A089I0Y6,EC:3.1.3.3/0.2126,NaN,NaN,3.1.3.3,0.2126


In [21]:
for i in range(len(output_csvs)):
    try:
        tophit = list(parse_annot(output_csvs[i]).value_counts("ec", normalize = True).reset_index().iloc[0])
        confidence = parse_annot(output_csvs[i]).groupby("ec")["confidence_ec1"].mean()[tophit[0]]
        print(" ".join([str(x) for x in tophit]))
        print(confidence)
    except FileNotFoundError:
        print("waiting")

3.1.3.3 0.6237623762376238
0.025163492063492065
3.1.3.73 0.405
0.07241358024691358
2.7.1.2 0.4
0.2324475
2.7.1.40 0.995
0.8436477386934673
2.7.1.40 1.0
0.9159860000000001
2.7.1.147 0.995
0.9531381909547738
2.7.1.1 1.0
0.9427540000000001
2.7.1.1 0.9620253164556962
0.6703986842105263
5.4.2.4 0.76
0.8408065789473684
5.4.2.11 0.86
0.9583936046511629
1.2.7.6 1.0
0.875137
2.7.1.1 1.0
0.9782700854700854
5.3.1.9 0.9642857142857143
0.23710185185185184
2.7.1.56 0.995
0.7558120603015075
2.7.1.11 0.54
0.8777611111111111
2.7.1.11 0.91
0.9819049450549451
4.1.2.13 0.595
0.14674453781512603
2.7.1.63 0.98
0.7433198979591836
4.2.1.11 0.995
0.9380341708542714
4.2.1.11 1.0
0.9602405
2.2.1.11 0.5596330275229358
0.6056770491803278
5.4.2.12 0.99
0.8026944444444445
2.7.2.3 0.985
0.9285720812182741
2.7.2.3 1.0
0.9914010000000001
2.7.1.90 0.8888888888888888
0.8944
2.7.1.11 0.595
0.9769957983193277
4.1.2.13 0.685
0.7888167883211679
4.1.2.13 0.76
0.8887657894736842
2.2.1.10 0.955
0.9603486910994765
5.3.1.1 0.97
0

In [42]:
output_csvs = [f"/data/luojaa/CLEAN/app/results/inputs/{file}_maxsep.csv" for file in ordered_infiles]
for i in range(len(ordered_infiles)):
    data_pfam, data_EC, data_GO = parse_annot(output_tsvs[i])
    kog = ordered_kogs[i]
    desc = kog_functions[kog_functions.KOGID == kog].FUNCTION.iloc[0]
    # print(output_tsvs[i])
    # print(kog)
    # print(desc)
    try:
        #print(" ".join([str(x) for x in list(data_EC.value_counts("predicted_label", normalize = True).reset_index().iloc[0,:])]))
        print(data_EC.value_counts("description").reset_index().description.iloc[0].strip("."))
    except IndexError:
        print(f"None")
    asdf
        
    #print(data_EC.value_counts("description"))

FileNotFoundError: [Errno 2] No such file or directory: '/data/luojaa/CLEAN/app/results/inputs/M00001.K15634_euk.tsv'